In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

import re
import time
import os
import numpy as np
import ast

ModuleNotFoundError: No module named 'selenium'

In [2]:
#get urls for meta archetypes
def getUrls(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    C = soup.find_all(class_='hover_tr')
    A = []
    for c in C:
        c = str(c)
        a = []
        if 'archetype' in c:
            c = c.split('meta')
            c[0] = c[0].split('a=')[1].split('&')[0]
            c[1] = c[1].split('>')[1].split('<')[0]
            A.append(c)
    format_ = url.split('f=')[1].split('&')[0]
    f = '&f='+format_+'&meta='+url.split('meta=')[1]
    link = ['https://www.mtgtop8.com/archetype?a=',f]
    for a in A:
        a[0] = link[0]+a[0]+link[1]
    return A
#get decklist urls from current page
def getDecks(B):
    C = soup.find_all(class_='hover_tr')
    for c in C:
        c = str(c)
        if 'event' in c:
            c = c.split()
            for b in c:
                if 'href' in b:
                    if 'event' in b:
                        if 'd=' in b:
                            b0 = b.split('e=')[1].split('&')[0]
                            b1 = b.split('d=')[1].split('&')[0]  
                            link = 'https://www.mtgtop8.com/event?e='+b0+'&d='+b1+'&f=MO'
                            B.append(link)
#Get decklists for all archetypes and save as csv
def decklists(A,s):
    df = pd.DataFrame()
    for a in A:
        df=df.append(pd.Series(data=[a[0],a[1],None]),ignore_index=True)
    for index,row in df.iterrows():
        if row[2]== np.nan or row[2]==None:
            url = row[0]
            time.sleep(2)
            driver = webdriver.Chrome('./chromedriver')
            driver.get(url)
            time.sleep(2)
            driver.find_element_by_xpath('/html/body/div[6]/div[2]').click()
            time.sleep(2)
            B = []
            flag = True
            i = 3
            while(flag):
                try:
                    b = '/html/body/div[3]/div/table/tbody/tr/td[2]/form[2]/table/tbody/tr/td/table/tbody/tr/td['+str(i)+']/div'
                    time.sleep(5)
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    getDecks(B)
                    button = driver.find_element_by_xpath(b)
                    button.click()
                    time.sleep(5)
                    i+=2
                except:
                    flag=False
            time.sleep(5)
            driver.quit()
            df.iloc[index,2] = B
    df.to_csv(s)

def getCardDictFromDecklistUrl(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    T = []
    for i in soup.find_all(name='td',class_=['G14','O13']):
        T.append(i.text)
    decklist = dict()
    types = ['LANDS','CREATURES','INSTANTS and SORC.','OTHER SPELLS','SIDEBOARD','COMMANDER']
    for type_ in types:
        decklist[type_] = []
    for t in T:
        for ty in types:
            if ty in t:
                cur_ty = ty
        if cur_ty not in t:
            decklist[cur_ty].append(t)
    return decklist

In [3]:
#Find all archetype URLS for modern 2018
# url = 'https://www.mtgtop8.com/format?f=MO&meta=163'
# A = getUrls(url)
# decklists(A,'modernMetaDecks2018.csv')


In [4]:
#Find all archetype URLS for modern 2019
# url = 'https://www.mtgtop8.com/format?f=MO&meta=183'
# A = getUrls(url)
# decklists(A,'modernMetaDecks2019.csv')


In [5]:
#Find all archetype URLS for modern 2020
# url = 'https://www.mtgtop8.com/format?f=MO&meta=200'
# A = getUrls(url)
# decklists(A,'modernMetaDecks2020.csv')


In [6]:
#Find all archetype URLS for commander 2018
# url = 'https://www.mtgtop8.com/format?f=EDH&meta=166'
# A = getUrls(url)
# decklists(A,'commandermetadecks2018.csv')


In [7]:
#Find all archetype URLS for commander 2019
# url = 'https://www.mtgtop8.com/format?f=EDH&meta=181'
# A = getUrls(url)
# decklists(A,'commandermetadecks2019.csv')


In [8]:
#Find all archetype URLS for commander 2020
# url = 'https://www.mtgtop8.com/format?f=EDH&meta=198'
# A = getUrls(url)
# decklists(A,'commandermetadecks2020.csv')


In [9]:
#bring in deck csvs
decklists = !ls *20*.csv
decklists

['fish: No matches for wildcard “*20*.csv”. See `help expand`.',
 'ls *20*.csv',
 '   ^']

In [10]:
#read csvs into a dict of dataframes
D = dict()
for d in decklists:
    D[d] = pd.read_csv(d,index_col=False)

#delete accidential index column
for d in D:
    D[d].drop(columns='Unnamed: 0',inplace=True)
    D[d].columns=[0,1,2]
    D[d][3] = None

NameError: name 'pd' is not defined

In [11]:
#read decklists into dataframe
for d in D:
    for index, row in D[d].iterrows():
        try:
            row[2] = ast.literal_eval(row[2])
        except:
            pass
        if row[3]==None or type(row[3])==float:
            row[3] = []
            for deck in row[2]:
                decklist = getCardDictFromDecklistUrl(deck)
                row[3].append(decklist)

In [12]:
# #download decklist files from decklist page and read into csv
# for d in D:
#     for index,row in D[d].iterrows():
#         try:
#             row[2] = ast.literal_eval(row[2])
#             !rm *.txt
#         except:
#             pass
        
#         if row[3]== None or type(row[3])==float:
#             row[3] = []
#             for url in row[2]:
#                 try:
#                     driver = webdriver.Chrome('./chromedriver')
#                     driver.get(url)
#                     time.sleep(2)
#                     driver.find_element_by_xpath('/html/body/div[5]/div/table/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/div/a[1]').click()
#                     time.sleep(2) 
#                     driver.quit()
#                     time.sleep(2)
#                     a = !ls *txt
#                     deck = []
#                     time.sleep(2)
#                     with open(a[0],'r') as f:
#                         deck.append(f.read())
#                     time.sleep(2)
#                     deck = deck[0].split('\n')
#                     deck = [d.split(' ') for d in deck]
#                     deck = [(d[0],' '.join(d[1:])) for d in deck]
#                     row[3].append(deck)
#                     !rm *.txt
#                     time.sleep(2)
#                 except:
#                     row[3].append(None)
#                     !rm *.txt

# for d in D:
#     D[d].to_csv('deckList'+d)